In [1]:
import pymysql
from langchain_community.llms import Ollama
from langchain_ollama import OllamaLLM  # Updated import

# Database connection settings 
DB_CONFIG = {
    "host": "localhost",
    "user": "root",
    "password": "12345678",
    "database": "10pearls",
    "port": 3306
}

# Initialize the LLM
local_model = "phi3.5"  
llm = OllamaLLM(model=local_model, temperature=0.1)

# Database schema context for LLM
DB_SCHEMA_CONTEXT = """
train_data schema:

1 - age: INT - Age of the client (numeric)

2 - job: VARCHAR(255) - Type of job (categorical: "admin.", "unknown", "unemployed", "management", "housemaid", "entrepreneur", "student", "blue-collar", "self-employed", "retired", "technician", "services")

3 - marital: VARCHAR(255) - Marital status (categorical: "married", "divorced", "single"; note: "divorced" means divorced or widowed)

4 - education: VARCHAR(255) - Level of education (categorical: "unknown", "secondary", "primary", "tertiary")

5 - default: VARCHAR(255) - Has credit in default? (binary: "yes", "no")

6 - balance: FLOAT - Average yearly balance, in euros (numeric)

7 - housing: VARCHAR(255) - Has housing loan? (binary: "yes", "no")

8 - loan: VARCHAR(255) - Has personal loan? (binary: "yes", "no")

9 - contact: VARCHAR(255) - Contact communication type (categorical: "unknown", "telephone", "cellular")

10 - day: INT - Last contact day of the month (numeric)

11 - month: VARCHAR(255) - Last contact month of the year (categorical: "jan", "feb", "mar", …, "nov", "dec")

12 - duration: INT - Last contact duration, in seconds (numeric)

13 - campaign: INT - Number of contacts performed during this campaign for this client (numeric, includes last contact)

14 - pdays: INT - Number of days that passed since the client was last contacted from a previous campaign (numeric; -1 means client was not previously contacted)

15 - previous: INT - Number of contacts performed before this campaign for this client (numeric)

16 - poutcome: VARCHAR(255) - Outcome of the previous marketing campaign (categorical: "unknown", "other", "failure", "success")

17 - y: VARCHAR(255) - Target outcome of the campaign, i.e., whether the client subscribed to the product (binary: "yes", "no")
"""

#DB connection
def connect_db():
    try:
        conn = pymysql.connect(**DB_CONFIG)
        return conn
    except pymysql.Error as e:
        print(f"Error connecting to MySQL: {e}")
        return None

# Generate SQL query from natural language query
def generate_sql_from_natural_language(natural_language_query):
    prompt = f"Use the following database schema context to generate an SQL query for MySQL database:\n{DB_SCHEMA_CONTEXT}\nRequest: {natural_language_query}\nEnsure that your query is SIMPLE, CORRECT, and MYSQL COMPLIANT."
    
    try:
        response = llm.invoke(prompt)
        
        # Ensure the response is structured correctly
        if "```sql" in response:
            sql_query = response.split("```sql")[1].split("```")[0].strip()
        else:
            sql_query = response.strip()
        
        # Strip unnecessary comments or extra text
        sql_query = sql_query.split("--")[0].strip()
        
        return sql_query
    except Exception as e:
        print(f"Error invoking LLM: {e}")
        return ""

# Run SQL query and return results
def execute_sql_query(conn, sql_query):
    cursor = conn.cursor()
    try:
        cursor.execute(sql_query)
        result = cursor.fetchall()
        columns = [desc[0] for desc in cursor.description]
        return columns, result
    except pymysql.Error as e:
        print(f"Error executing SQL query: {e}")
        return None, None
#Generate natural language report from results
def generate_report_from_results(columns, result, user_query):
    prompt = f"""
    Given the results of a SQL query and the user's question, summarize the main findings in a clear and concise manner.
    Focus only on the relevant information based on the SQL query result.

    columns: {columns}
    User's Query: {user_query}
    SQL Query Result: {result}

    Please provide a concise summary in plain English.
    """
    try:
        report = llm.invoke(prompt)
        return report
    except Exception as e:
        print(f"Error generating report: {e}")
        return "Error generating report."

#for debuging print sql query results
def print_results(columns, result):
    print("Query Results:")
    print("-" * 40)
    print(" ".join(f"{col:<20}" for col in columns))
    print("-" * 40)
    for row in result:
        print(" ".join(f"{str(val):<20}" for val in row))

def main():
    while True:
        natural_language_query = input("Enter your natural language query (or 'quit' to exit): ")
        if natural_language_query.lower() == 'quit':
            break
        print(f"Query: {natural_language_query}")
        
        sql_query = generate_sql_from_natural_language(natural_language_query)
        
        print(f"Generated SQL Query: {sql_query}")

        # Connect to the database and execute the query
        conn = connect_db()
        if conn:
            columns, result = execute_sql_query(conn, sql_query)
            conn.close()

            if columns and result:
                print_results(columns, result)
                
                # Generate and display report
                report = generate_report_from_results(columns, result, natural_language_query)
                print("Generated Report:\n", report)
            else:
                print("No results found.")
        else:
            print("Failed to connect to the database.")

if __name__ == "__main__":
    main()


Query: how many people have the job admin
Generated SQL Query: SELECT COUNT(*) AS number_of_admin_jobs
FROM train_data
WHERE job = 'admin.';
Query Results:
----------------------------------------
number_of_admin_jobs
----------------------------------------
5171                
Generated Report:
 The SQL query result indicates that there are currently 5,171 individuals who hold an administrative position within the dataset examined by the database search. This figure represents all people with jobs classified as 'admin' across various records or entries analyzed during this specific data retrieval process.
Query: what is the average age
Generated SQL Query: SELECT AVG(age) AS AverageAge FROM train_data;
Query Results:
----------------------------------------
AverageAge          
----------------------------------------
40.9362             
Generated Report:
 The main finding from the query result indicates that the calculated average age of individuals (or entities being analyzed, dep